<a href="https://colab.research.google.com/github/vaanicodes/MC432-Project/blob/main/Neuro_Fuzzy_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Dynamic Clustering

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


def clustering(df,count):
    df.sort_values(by=df.columns[count],inplace=True)
    df.reset_index(drop=True, inplace=True)
    Target = list(df['class'])
    q = df['class'][0]
    Cluster_form = list()
    cl = list()
    for i in range(0,len(Target)):
        if Target[i] == q:
            cl.append(df[df.columns[count]][i])
        else:
            q = df['class'][i]
            Cluster_form.append(cl)
            cl = list()
            cl.append(df[df.columns[count]][i])
    Cluster_form.append(cl)
#clustering of feature values
    #print("CLuster",z)
    s1 = 0
    for v in Cluster_form: 
        s1+=len(v)
    Threshhold = s1/len(Cluster_form) 
    #print("s1",s1)
    #print(len(z))
    #print("t",Threshhold)
    return centroid(Cluster_form,Threshhold)


def centroid(Cluster_form,Threshhold):
# list as a input for centroid function
    def cen(i):
        for i in Cluster_form:
            ci = sum(i)/len(i)
            return ci
    
    #print("len",len(z))

#cluster are combine with other cluster based on less distance between centroid of clusters
    for i in Cluster_form:
        if len(i) < Threshhold:
            if Cluster_form.index(i) == 0:
                for t in i:
                    Cluster_form[Cluster_form.index(i)+1].append(t)
            elif Cluster_form.index(i) == len(Cluster_form)-1:
                for t in i:
                    Cluster_form[Cluster_form.index(i)-1].append(t)
            elif (cen(i)-cen(Cluster_form[Cluster_form.index(i)-1]))<(cen(i)-cen(Cluster_form[Cluster_form.index(i)+1])): 
                for t in i:
                    Cluster_form[Cluster_form.index(i)-1].append(t)     
            elif (cen(i)-cen(Cluster_form[Cluster_form.index(i)-1]))>(cen(i)-cen(Cluster_form[Cluster_form.index(i)+1])):
                for t in i:
                    Cluster_form[Cluster_form.index(i)+1].append(t)

#clusters which have member in cluster less than Fth are eliminated
    new = list()
    for i in Cluster_form:
        if len(i) > Threshhold:
            new.append(i)
    #print("new",len(new))
    #print("newnn",new) #list with newly formed cluster after elimination

#mean is calculated
    mean = list()
    for i in new:
        mean.append(sum(i)/len(i))  
    #print(mean)
    #print('\n')

#sd is calculated
    SD = list()
    for i in new:
        SD.append(np.std(i)) 
    #print(SD)
    return mean,SD


def Dynamic_clustering(df):       
    count=0   
    MEAN=[]
    Std_deviation=[]
    while count<len(df.columns)-1:
        a,b=clustering(df,count)
        MEAN.append(a)
        Std_deviation.append(b)
    
    #print(len(df.columns))
        count+=1
    #print(MEAN)
    #print(len(MEAN))
    #print(Std_deviation)
    return MEAN,Std_deviation


# Fuzzification

In [2]:
import numpy as np

class Fuzzification:
    def __init__(self, row, mean, sd):
        self.row = row #each row with multiple features
        self.mean = mean #means from dynamic clustering for each features
        self.sd = sd #sds from dynamic clustering for each features
        
    def normal_density(self, x, mean, sd):
        if sd == 0:
            if mean == x:
                return 1
            else:
                return 0
        return (1/(sd*(2*np.pi))**(0.5))* np.exp(-0.5*((x-mean)/sd)**(2))
    
    def fuzzify(self):
        if len(self.row) != len(self.mean):
            raise Exception('Need means for each feature')
        if len(self.row) != len(self.sd):
            raise Exception('Need sd for each feature')
        fuzzy_output = []    
        for i in range(len(self.row)):           
            clusters = len(self.mean[i])
            output = np.zeros((clusters))
            densities = []
            for j in range(clusters):
                density = self.normal_density(self.row[i], self.mean[i][j], self.sd[i][j])
                densities.append(density)
        
            index = densities.index(max(densities))
            output[index] = 1
            output = output.tolist()
            output = [int(x) for x in output]
            fuzzy_output.append(output)
            #fuzzy_output.append()
        return fuzzy_output

# Neural Weights Function

In [3]:
import pandas as pd
#import matplotlib.pyplot as plt
import numpy as np
import random
from collections import Counter

#plt.close("all")

#------------------------------------------------------------------------------------------------------
# Forecasting Algorith Developed from the Neuro - Fuzzy System
# Authors: Phichit Napook and Narissara Eiamkanitcaht
#       Chiang Mai University,Thailand
# Copyright 2015 © 2015 ACM 978-1-4503-3575-1/15/09 $15.00.
# http://dx.doi.org/10.1145/2800835.2800983
#------------------------------------------------------------------------------------------------------

# Testing Comment for Compilation
#print('Starting')

#------------------------------------------------------------------------------------------------------
#    Neural Network
#------------------------------------------------------------------------------------------------------

weights  = np.random.rand(2,3)
inputs = [[3,3,3],[-3,-3,-3],[4,4,-4]]
biases = np.random.rand(2)
expectedOutput = [1,0,1]

def my_neuron(weightList, inputList, bias, my_function):
	dotproduct = np.dot(weightList,inputList)
	fullproduct = dotproduct + bias
	return my_function(fullproduct)

def my_sigmoid(value):
	value2 = value * -1
	sigmoid = 1 / (1 + np.exp(value2))
	return sigmoid

def my_derivative_sigmoid(value):
	sigmoid = my_sigmoid(value) * (1 - my_sigmoid(value))
	return sigmoid

def sigmoid_neuron(weightList, inputList, bias):
	return my_neuron(weightList, inputList, bias, my_sigmoid)

def singleInput_neuron_layer(EveryNeurons_weightList, inputList, EveryNeurons_bias, Individual_neuron):
	full_output = []
	for weightList,bias in zip(EveryNeurons_weightList, EveryNeurons_bias):
		single_output = Individual_neuron(weightList, inputList, bias)
		full_output.append(single_output)
	return full_output

def neuron_layer(EveryNeurons_weightList, Every_inputList, EveryNeurons_bias, Individual_neuron):
	full_output = []
	for inputList in Every_inputList:
		single_output = singleInput_neuron_layer(EveryNeurons_weightList, inputList, EveryNeurons_bias, Individual_neuron)
		full_output.append(single_output)
	return full_output

def simple_error(expectedValue, actualValue):
	diff = (expectedValue - actualValue) * (expectedValue - actualValue)
	return diff

def layer1_derivative_chain(weightList, expectedValue, actualValue, Adenduminputs, maybeBias):
	dotproduct = np.dot(weightList,Adenduminputs)
	fullproduct = dotproduct + maybeBias
	full_output = []
	sigmoid_derivative = my_derivative_sigmoid(fullproduct)
	#for w_weight in weightList:
	for in_input in Adenduminputs:
		per_weight_derivative = -0.5 * sigmoid_derivative * (expectedValue - actualValue) * in_input #in_input #w_weight
		full_output.append(per_weight_derivative)
	return full_output

def layer1_bias_chain(weightList, expectedValue, actualValue, Adenduminputs, maybeBias):
	dotproduct = np.dot(weightList,Adenduminputs)
	fullproduct = dotproduct + maybeBias # simple_error(expectedValue, actualValue)
	return 0 #my_derivative_sigmoid(fullproduct)

def layer1_weight_adjustment(weightList, bias, expectedValue, actualValue, Adenduminputs):
	weight_adjustments = layer1_derivative_chain(weightList, expectedValue, actualValue, Adenduminputs, bias)
	bias_adjustment = layer1_bias_chain(weightList, expectedValue, actualValue, Adenduminputs, bias)
	new_weightList = np.subtract(weightList, weight_adjustments) #np.add for opposite affect subtract
	new_bias = bias + bias_adjustment
	return [new_weightList, new_bias]

def training_attempt(weightList, bias, inputs, sigmoid_neuron, expectedValue):
	actualValue = singleInput_neuron_layer(weightList,inputs,bias,sigmoid_neuron)
	return layer1_weight_adjustment(weightList, bias, expectedValue, actualValue, inputs)

def multiple_training_attempts(weightLists, biases, inputLists, sigmoid_neuron, expectedValues):
	new_weightLists = []
	new_biasList = []
	actualValues = neuron_layer(weightLists,inputLists,biases,sigmoid_neuron)
	for weightList,bias in zip(weightLists, biases):
		for inputs, expectedValue, actualValue in zip(inputLists, expectedValues, actualValues):
			adjustments = layer1_weight_adjustment(weightList, bias, expectedValue, actualValue[0], inputs)
			weightList = adjustments[0]
			bias = adjustments[1]
		new_biasList.append(bias)
		new_weightLists.append(weightList)
	return [new_weightLists, new_biasList]

def bulk_training(weightLists, biases, inputLists, sigmoid_neuron, expectedValues):
	for x in range(30):
		listofnewweightsbiases = multiple_training_attempts(weightLists, biases, inputLists, sigmoid_neuron, expectedValues)
		weightLists = listofnewweightsbiases[0]
		biases = listofnewweightsbiases[1]
	return listofnewweightsbiases

def sigmoid_training(weightLists, biases, inputLists, expectedValues):
	return bulk_training(weightLists, biases, inputLists, sigmoid_neuron, expectedValues)

def sigmoid_training_special(inputLists, expectedValues):
	inputlength = len(inputLists[0])
	weightLists = np.random.rand(1,inputlength)
	biases = [0]
	return bulk_training(weightLists, biases, inputLists, sigmoid_neuron, expectedValues)

def neruotest():
	print("Starting Weights and Answer Estimation")
	print([weights, biases])
	print(neuron_layer(weights,inputs,biases,sigmoid_neuron))
	listofnewweightsbiases = bulk_training(weights,biases, inputs, sigmoid_neuron, expectedOutput)
	print("Ending Weights and Answer Estimation")
	print(listofnewweightsbiases)
	print(neuron_layer(listofnewweightsbiases[0],inputs,listofnewweightsbiases[1],sigmoid_neuron))

#neruotest()

#------------------------------------------------------------------------------------------------------
#    Quadratic Function
#------------------------------------------------------------------------------------------------------

#training_weights = weights

#Based on idea that the paper talks about the length of the weights.
#maxlength = len(training_weights[0]) # As in the total number of weights that are in a singular node

#x_0 = 0 #Lowest weight is 0 in sigmoid
#x_1 = random.randint(1,maxlength)#np.random.choice(training_weights)#np.random.rand(1)[0] #Random weight in sigmoid between 0 and 1
#x_2 = maxlength#max(training_weights) #Highest weight is 1 in sigmoid
#x_3 = 0 # Not Initialized Yet.

# A, if a weight set applied to an input set is equal to 1, else 0

def calc_x_uninitialized(x,training_data,training_weights):
	calc_sum = 0
	for training_weight in training_weights:
		#Big_A = 1
		for training_item in training_data:
			Big_A = 1
			j = 0
			while ((Big_A == 1) and (j < x) and (j < len(training_weights[0]))) :
				if (training_item[j] * training_weight[j] >= .9):
					Big_A = 1
				else:
					Big_A = 0
				j += 1
			calc_sum += Big_A
	return calc_sum

def calc_x3_internal(x_0,x_1,x_2, training_data, training_weights):
	def f(x):
		return calc_x_uninitialized(x,training_data,training_weights)
	top_half = f(x_2)*((x_0**2)-(x_1**2)) + f(x_0)*((x_1**2)-(x_2**2)) + f(x_1)*((x_2**2)-(x_0**2))
	bottom_half = 2*f(x_2)*(x_0-x_1) +  2*f(x_0)*(x_1-x_2) +  2*f(x_1)*(x_2-x_0)
	if bottom_half == 0:
		return top_half/.1
	return top_half/bottom_half

def calc_x(training_data, training_weights):
	maxlength = len(training_weights[0])
	x_0 = 1 #Lowest weight is 0 in sigmoid
	x_1 = random.randint(1,maxlength)#np.random.choice(training_weights)#np.random.rand(1)[0] #Random weight in sigmoid between 0 and 1
	x_2 = maxlength#max(training_weights) #Highest weight is 1 in sigmoid
	x_3 = 0
	def f(x):
		return calc_x_uninitialized(x,training_data,training_weights)
	x_3 = calc_x3_internal(x_0,x_1,x_2, training_data, training_weights)
	f_x_3 = f(x_3)
	f_x_2 = f(x_2)
	f_x_0 = f(x_0)
	f_x_1 = f(x_1)
	while ((f_x_0- f_x_1) >= 0):
		#do all this
		if (f_x_3 == f_x_2):
			x_2 = x_3
			f_x_2 = f_x_3

		#Iterate_random process of X_1 ???
		elif (f_x_3 < f_x_2):
			x_0 = x_3
			f_x_2 = f_x_3
		print("I'm here")

		x_1 = x_3
		f_x_1 = f_x_3
		top_half = f_x_2*((x_0**2)-(x_1**2)) + f_x_0*((x_1**2)-(x_2**2)) + f_x_1*((x_2**2)-(x_0**2))
		bottom_half = 2*f_x_2*(x_0-x_1) +  2*f_x_0*(x_1-x_2) +  2*f_x_1*(x_2-x_0)
		if bottom_half == 0:
			x_3 = top_half/.1
		else:
			x_3 = top_half/bottom_half
		f_x_3 = f(x_3)
	print(f_x_3)
	return x_1
#print(calc_x(x_0,x_1,x_2, inputs, weights))
# Use X_1 to creat the binary clasification somehow???


def important_feature_selection(listofweights):
	newlist = listofweights[0]
	returnlist = []
	count = 0
	for i in newlist:
		returnlist.append((i,count))
		count += 1
	newerlist = sorted(returnlist, reverse = True)
	return newerlist

def important_feature_selection_last(listofweights):
	newlist = listofweights[0]
	returnlist = []
	count = 0
	for i in newlist:
		returnlist.append((i,count))
		count += 1
	newerlist = sorted(returnlist, reverse = True)
	return newerlist[-10:]

#classifier(x_0,x_1,x_2, new_inputs,  ,trained_weights)
weights  = np.random.rand(2,3)
inputs = [[3,3,3],[-3,-3,-3],[4,4,-4]]
biases = np.random.rand(2)
expectedOutput = [1,0,1]

def fulltest():
	print("Starting Weights and Answer Estimation")
	print([weights, biases])
	#print(neuron_layer(weights,inputs,biases,sigmoid_neuron))
	listofnewweightsbiases = sigmoid_training_special(inputs, expectedOutput)
	print("Ending Weights and Answer Estimation")
	print(listofnewweightsbiases)
	
	#classifier_x = calc_x(inputs, listofnewweightsbiases[0])
	classifier_x = important_feature_selection(listofnewweightsbiases[0])
	print("Classifier Rule Estimation")
	print(classifier_x)

	#somevalue = 3
	def my_rule(x):
		if my_sigmoid(x) > classifier_x:
			return 1
		else:
			return 0
	#print(my_rule(somevalue))



# Quadrification

In [4]:
import math

def thefunction(x,sortedlistofweights,fuzzyvalues): # the list of weights have to be sorted
    if x > len(sortedlistofweights):
        return "Input not in range!"
    if x == 0:
        return 0
    sum = 0
    for item in fuzzyvalues:
        j = 0
        while j < x:
            if item[sortedlistofweights[j][1]] == 1 and sortedlistofweights[j][0] > 0.98:
                A = 1
            else:
                A = 0
            j += 1
        sum = sum + A
    return sum




gr = (math.sqrt(5) - 1) / 2

def gss(f, a, b, tol = .01):
    c = b - (b - a) * gr
    d = a + (b - a) * gr
    while abs(b - a) > tol:
        if f(c) < f(d):
            b = d
        else:
            a = c

        # We recompute both c and d here to avoid loss of precision which may lead to incorrect results or infinite loop
        c = b - (b - a) * gr
        d = a + (b - a) * gr

    return (b + a) / 2


# Testing on mesothelioma data

In [5]:
from sklearn.model_selection import train_test_split
meso_data = pd.read_excel("/content/Mesothelioma data set.xlsx", header = 0)
data = meso_data[meso_data.columns[1:len(meso_data.columns)-1]]
column_list= list(data.columns)
for name in column_list:
    data = data.rename(columns={name:"feat"+str(column_list.index(name))})

#targets classes 
t=  meso_data[['class of diagnosis']].replace([1,2],[0,1])

#this is dataframe with cleaned data which has the features and the class(either 0(M) or 1(B))
df = pd.concat([data, t], axis=1)
df = df.rename(columns={"class of diagnosis":"class"})
#print(df.sample(n=5))

train, test = train_test_split(df, test_size=0.2, random_state=42, shuffle=False)

#desired_t = [[1,0],[0,1]]
#targets in list format for neural netwrork eg. [0,1] for class being B [1,0] for class being M
train_t = [int(x) for x in train.values[:,-1]]

test_t = [int(x) for x in test.values[:,-1]]



#means and sds from dynamic clustering
MEAN,Std_deviation = Dynamic_clustering(train)

#values after fuzzification.. it's a list of list.
fuzzy_values = []
train_f = train.iloc[: , :-1]

for index, row in train_f.iterrows():
    f = Fuzzification(row, MEAN, Std_deviation)
    fuzzy_val = f.fuzzify()
    fuzzy_values.append(sum(fuzzy_val, []))
    
fuzzy_values_test = []
test_f = test.iloc[: , :-1]

for index, row in test_f.iterrows():
    f = Fuzzification(row, MEAN, Std_deviation)
    fuzzy_val = f.fuzzify()
    fuzzy_values_test .append(sum(fuzzy_val, []))



# Training the network to pick out best weights
listofnewweightsbiases = sigmoid_training_special(fuzzy_values, train_t)
#sorting the weights
sortedweights = important_feature_selection(listofnewweightsbiases[0])


def g(x):
	return thefunction(x,sortedweights,fuzzy_values)
num = gss(g,0,20)

#print(train_t)
#print("\n")
#print(fuzzy_values_test)
#result = neuron_layer(listofnewweightsbiases[0],fuzzy_values_test,listofnewweightsbiases[1],sigmoid_neuron)
#print(result)

# Finding the accuracy of our rule creation.

def test_nerual_accuracy(listofresults,listofactual):
	count = 0
	for result, actual in zip(listofresults, listofactual):
		if result[0] == actual:
			count += 1
	return count/len(listofresults)

# Result is only for guaging neural network accuracy 
# MAKE RULES METHOD
# [1,0,1,0,1,0....]
# n specifies the no: of important feature chosen
print(sortedweights[:5])
print(fuzzy_values_test[0])
def myrules(input,n):
	for a in sortedweights[:n]:
		#print(a[1])
		if input[a[1]] == 1:
			return 1
	return 0

def applyrules(all_inputs,n):
	mynewlist = []
	for inputs in all_inputs:
		mynewlist.append(myrules(inputs,n))
	return mynewlist

def test_rule_accuracy(listofresults,listofactual):
	count = 0
	for result, actual in zip(listofresults, listofactual):
		if result == actual:
			count += 1
	return count/len(listofresults)

for i in [3,5,8,10,15,int(num)]:
	print("The accuracy of our rules  with top" , i, "features is:", test_rule_accuracy(applyrules(fuzzy_values_test,i), test_t))

[(0.9999419255923946, 113), (0.9994564216460736, 538), (0.9987715666161301, 948), (0.9982727770990693, 831), (0.9942629105186784, 366)]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,